## Introduction
* Part1: Prepare data to visualize the labeled stock price(By sentiment result: delay two days)
* Part2: Realize the LSTM model


## Part1

### Import packages

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix


2022-12-04 15:37:56.968894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [18]:
Twitter_stock1=pd.read_csv("../Data/combind_sen_stock.csv")
Twitter_stock2=pd.read_csv("../Data/Combined_Delayed_2Days.csv")

Twitter_stock2

,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,Decrease
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,Increase
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,Decrease
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,Decrease
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,Increase
...,...,...,...,...,...,...
1415,2018-09-07,type peopl world understand binari amp will s...,0.492857,0.567857,positive,Decrease
1416,2014-09-08,excel explan diff betw get orbit v space sugge...,0.250000,0.333333,positive,Decrease
1417,2020-09-08,a la guerr comm à la guerr haha and start twee...,0.100000,0.400000,positive,Decrease
1418,2014-09-09,meet build call just crazi japanes game show ...,-0.400000,0.400000,negative,Increase


In [17]:
df_lh=pd.read_csv("../Data/TSLA_labeled.csv")
df_lh

,Date,Open,High,Low,Close,Adj Close,Volume,Label
0,2011-12-01,2.171333,2.266000,2.132000,2.173333,2.173333,15453000,Increase
1,2011-12-02,2.188667,2.246000,2.160000,2.220000,2.220000,12042000,Increase
2,2011-12-05,2.235333,2.333333,2.228667,2.294667,2.294667,17401500,Increase
3,2011-12-06,2.280000,2.332000,2.268667,2.324667,2.324667,14277000,Increase
4,2011-12-07,2.308667,2.326000,2.253333,2.279333,2.279333,10114500,Decrease
...,...,...,...,...,...,...,...,...
2353,2021-04-12,228.566666,234.933334,227.363327,233.993332,233.993332,87407100,Increase
2354,2021-04-13,237.566666,254.333328,236.886673,254.106674,254.106674,133958400,Increase
2355,2021-04-14,256.899994,260.263336,242.676666,244.076660,244.076660,147052200,Decrease
2356,2021-04-15,247.699997,247.896667,240.436661,246.283340,246.283340,83546700,Decrease


In [19]:
twitter_stock_2=Twitter_stock2.merge(df_lh)
twitter_stock_2.head()


,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days,Open,High,Low,Close,Adj Close,Volume,Label
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,Decrease,2.258000,2.266000,2.225333,2.235333,2.235333,13837500,Decrease
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,Increase,22.293333,23.025999,22.119333,22.997999,22.997999,90846000,Increase
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,Decrease,32.119331,32.329334,31.580000,31.876667,31.876667,194392500,Decrease
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,Decrease,1.841333,1.892000,1.820000,1.882000,1.882000,10084500,Increase
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,Increase,14.267333,14.296667,13.533333,13.856667,13.856667,61371000,Decrease


In [20]:
label_line2=np.array(twitter_stock_2["High"])-np.array(twitter_stock_2["Low"])/2
twitter_stock_2[" half_H_L"]=label_line2
twitter_stock_2.head()

,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days,Open,High,Low,Close,Adj Close,Volume,Label,half_H_L
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,Decrease,2.258000,2.266000,2.225333,2.235333,2.235333,13837500,Decrease,1.153334
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,Increase,22.293333,23.025999,22.119333,22.997999,22.997999,90846000,Increase,11.966332
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,Decrease,32.119331,32.329334,31.580000,31.876667,31.876667,194392500,Decrease,16.539334
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,Decrease,1.841333,1.892000,1.820000,1.882000,1.882000,10084500,Increase,0.982000
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,Increase,14.267333,14.296667,13.533333,13.856667,13.856667,61371000,Decrease,7.530001


In [21]:
twitter_stock_2.to_csv("../Data/Twitter_stock_delay2.csv")

## Part2

### LSTM MODEL

In [5]:
df=pd.DataFrame([Twitter_stock1["Tweets_clean_more"],Twitter_stock2["Label_delayed_2Days"]]).T



In [10]:
Twitter_stock2["Year"]=[Twitter_stock2["Date"][i][0:4] for i in range(Twitter_stock2.shape[0])]
Twitter_stock2.head()


,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days,Year
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,Decrease,2013
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,Increase,2019
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,Decrease,2020
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,Decrease,2012
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,Increase,2016


In [13]:
Twitter_stock2.groupby(["Year"]).max()

,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days
Year,,,,,,
2011,2011-12-30,yeah realli mom attest not sure i handl char m...,0.6,0.708687,positive,No Change
2012,2012-12-26,ye unveil fulli function advanc prototyp model...,0.8,1.000000,positive,No Change
2013,2013-12-26,ye upper stage vent liquid oxygen creat fast m...,1.0,1.000000,positive,Increase
2014,2014-12-31,yeah that worth read what could possibl go wro...,0.9,1.000000,positive,Increase
2015,2015-12-22,yesterday mission space station light cloudi n...,1.0,1.000000,positive,No Change
2016,2016-12-30,yup ye ye pretti close area ratio vacuum mar a...,1.0,1.000000,positive,No Change
2017,2017-12-27,yeah sorri i miss thi would work kwh pack soft...,1.0,1.000000,positive,No Change
2018,2018-12-31,your cruel taunt cut deep deep but i love oh g...,0.8,0.900000,positive,No Change
2019,2019-12-31,you welcom glad help,0.9,1.000000,positive,No Change


In [6]:
label_encoder = LabelEncoder()

df["Label"] = label_encoder.fit_transform(df["Label_delayed_2Days"])



### Get the training and testing dataset

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df["Tweets_clean_more"], df["Label"], test_size=0.2)

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train shape:  (1137,)
X_val shape:  (285,)
y_train shape:  (1137,)
y_val shape:  (285,)


In [13]:
print('Train Class Proportion:\n', y_train.value_counts() / len(y_train) * 100)
print('\nValidation Class Proportion:\n', y_val.value_counts() / len(y_val) * 100)

Train Class Proportion:
 1    48.900616
0    48.460862
2     2.638522
Name: Label, dtype: float64

Validation Class Proportion:
 0    49.824561
1    46.315789
2     3.859649
Name: Label, dtype: float64


In [14]:


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df["Tweets_clean_more"])
xtrain= tokenizer.texts_to_sequences(X_train)
xtest= tokenizer.texts_to_sequences(X_val) 




In [15]:
vocab_size=len(tokenizer.word_index)+1


 
maxlen = 50

X_train_text = pad_sequences(xtrain, padding='post', maxlen=maxlen)
X_val_text = pad_sequences(xtest, padding='post', maxlen=maxlen)

print('X_train shape:', X_train_text.shape)
print('X_train shape:', X_val_text.shape)



X_train shape: (1137, 50)
X_train shape: (285, 50)


In [20]:
embedding_dim=50
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
model.summary()
model.fit(X_train_text,y_train, epochs=20, batch_size=16, verbose=False)



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 50)            368000    
                                                                 
 lstm_6 (LSTM)               (None, 50, 50)            20200     
                                                                 
 lstm_7 (LSTM)               (None, 10)                2440      
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                                 
 dense_6 (Dense)             (None, 3)                 33        
                                                                 
 dense_7 (Dense)             (None, 1)                 4         
                                                                 
Total params: 390,677
Trainable params: 390,677
Non-tr

In [22]:
loss, acc = model.evaluate(X_train_text, y_train, verbose=False)
print("Training Accuracy: ", acc)
loss, acc = model.evaluate(X_val_text, y_val, verbose=False)
print("Test Accuracy: ", acc)





Training Accuracy:  0.9639402031898499
Test Accuracy:  0.4385964870452881


In [52]:
ypred=model.predict(X_val_text)


ypred[ypred>0.5]=1 
ypred[ypred<=0.5]=0 

9/9 [==============================] - 0s 8ms/step


In [54]:
cm = confusion_matrix(y_val, ypred)
cm

array([[  0, 135,   0],
       [  0, 138,   0],
       [  0,  12,   0]])

In [59]:
print(classification_report(y_val,ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       135
           1       0.48      1.00      0.65       138
           2       0.00      0.00      0.00        12

    accuracy                           0.48       285
   macro avg       0.16      0.33      0.22       285
weighted avg       0.23      0.48      0.32       285



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Results:
The LSTM model show us that the predict result. 

